In [1056]:
import numpy as np

def read_data(filename):
    d = np.loadtxt(filename, delimiter=",", encoding='utf-8', dtype=str)
    new_cases = np.array(d[1:,1:], dtype=int)
    norm = new_cases / np.max(new_cases, axis=1).reshape((new_cases.shape[0],1))
    return {'new_cases': new_cases, 
    'norm_data': norm,
    'province_names': d[1:,0],
    'dates': d[0,1:]}

ข้อห้าม
เนื่องจากโจทย์ข้อนี้ต้องการทดสอบความเข้าใจในเนื้อหาเรื่อง numpy จึงกำหนดข้อห้ามด้งต่อไปนี้

    • ห้าม import package ใด ๆ เพิ่มเติม
    • ห้ามใช้คำสั่ง for, while, list comprehension
    (รวมถึงการใช้set/dict comprehension, map, reduce, ฟังก์ชันแบบ recursive ถ้ารู้ว่าคืออะไร)
    • ไม่เขียนคำสั่งใด ๆ นอกฟังก์ชัน แต่สามารถเขียนฟังก์ชันเพิ่มเติมได้
    • ไม่ใช้ตัวแปรใด ๆ ที่อยู่นอกฟังก์ชัน

ข้อแนะนำ

    • ศึกษาเพิ่มเติมเรื่อง broadcast ฟังก์ชัน transpose และการตั้งค่า axis ของอาเรย์สามมิติ
    • ถ้าไม่เขียนการทำงานใด ๆ ในฟังก์ชันใด อย่าลืมเขียนคำสั่ง return สักหนึ่งบรรทัดในฟังก์ชันนั้น
    • เขียนคำสั่งทดสอบต่าง ๆ ได้ในฟังก์ชัน main

In [1057]:
data = read_data('ass.csv')
data

{'new_cases': array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   2,   1,   1,   1,
           5,   5,   5],
        [  7,  20,  32,  35,  46, 156, 216,  95, 268, 185, 236, 137, 194,
         351, 409, 312],
        [  0,   0,   0,   0,   0,   1,   0,   0,   1,   1,   1,  13,   0,
          16,   5,   4],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   1,   0,
           1,   6,   1],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   2,   1,   2,   1,
           2,   1,   4],
        [  0,   0,   0,   2,   0,   0,   2,   3,   6,   5,   6,   9,  12,
          18,  18,   8],
        [  0,   0,   0,   0,   0,   0,   1,   0,   2,   3,   1,   8,   9,
           5,   6,   2],
        [  0,   0,   0,   0,   0,   0,   0,   1,   2,   6,   2,  10,   7,
          16,   7,   9],
        [  0,   0,   1,   7,   7,  18,  11,  14,  46, 101, 180,  92,  97,
          72,  98, 111],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   2,   0,   0,
           2,   0,   0

In [1058]:
def max_new_cases_date(DATA):
    ind = np.argmax(np.sum(DATA['new_cases'], axis = 0))
    return DATA['dates'][ind], np.sum(DATA['new_cases'],axis = 0)[ind]

In [1059]:
def max_new_cases_province(DATA, beg_date, end_date):
    ind_beg_date = np.where(DATA['dates'] == beg_date)[0][0]
    ind_end_date = np.where(DATA['dates'] == end_date)[0][0]

    caseByDate = DATA['new_cases'].take(indices = range(ind_beg_date, ind_end_date+1), axis = 1)

    ind = np.argmax(np.sum(caseByDate, axis = 1))
    return DATA['province_names'][ind], np.sum(caseByDate,axis = 1)[ind]

In [1060]:
def max_new_cases_province_by_dates(DATA):
    arr = np.ndarray( (3,16), dtype=object)
    arr[0] = DATA['dates']
    arr[1] = DATA['province_names'][np.argmax(DATA['new_cases'], axis = 0)]
    arr[2] = np.max(DATA['new_cases'], axis=0)
    return arr.transpose()

In [1061]:
def most_similar(DATA, province):
    province_ind = np.where(DATA['province_names'] == province)[0][0]
    sim = list(np.sum((DATA['norm_data'][province_ind] - DATA['norm_data'])**2, axis = 1))
    return DATA['province_names'][np.argmin(sim[:province_ind] + sim[province_ind+1:])+1]

In [1062]:
def most_similar_province_pair(DATA):
    prov = (np.add(DATA['province_names'].astype(object).reshape(-1,1), ' ') + DATA['province_names'].astype(object)).reshape(-1,1)

    a = np.sum(np.power(np.repeat(DATA['norm_data'], 77,axis = 0) - np.tile(DATA['norm_data'], (77,1)),2), axis=1)
    b = np.where(a==0,np.nan,a)

    np.where(b == np.nanmin(b))
    same_norm_diff_prov = np.mod(np.where(a==0),78)
    if np.sum(same_norm_diff_prov) != 0:

        ind = (np.take(np.where(a==0)[0],[np.where((np.mod(np.where(a==0),78) != 0))[1:]]))[0][0]
        np.put(b,[ind],np.take(a,[ind]))

    ans = str(np.random.choice(np.take(prov, np.where(b == np.nanmin(b)))[0])).split()

    return ans[0], ans[1]


In [1063]:
def most_similar_in_period(DATA, province, beg_date, end_date):
    ind_beg_date = np.where(DATA['dates'] == beg_date)[0][0]
    ind_end_date = np.where(DATA['dates'] == end_date)[0][0]

    normByDateProv = DATA['norm_data'].take(indices = range(ind_beg_date, ind_end_date+1), axis = 1)

In [1064]:
def main():
    print(max_new_cases_date(data))
    print(max_new_cases_province(data, '2021-04-10', '2021-04-13'))
    print(max_new_cases_province_by_dates(data))
    print(most_similar(data,'กรุงเทพมหานคร'))
    print(most_similar_province_pair(data))
    return

In [1065]:
main()

('2021-04-16', 1577)
('เชียงใหม่', 854)
[['2021-04-01' 'สมุทรสาคร' 11]
 ['2021-04-02' 'กรุงเทพมหานคร' 20]
 ['2021-04-03' 'กรุงเทพมหานคร' 32]
 ['2021-04-04' 'กรุงเทพมหานคร' 35]
 ['2021-04-05' 'นราธิวาส' 94]
 ['2021-04-06' 'กรุงเทพมหานคร' 156]
 ['2021-04-07' 'กรุงเทพมหานคร' 216]
 ['2021-04-08' 'นราธิวาส' 146]
 ['2021-04-09' 'กรุงเทพมหานคร' 268]
 ['2021-04-10' 'กรุงเทพมหานคร' 185]
 ['2021-04-11' 'กรุงเทพมหานคร' 236]
 ['2021-04-12' 'เชียงใหม่' 246]
 ['2021-04-13' 'เชียงใหม่' 251]
 ['2021-04-14' 'กรุงเทพมหานคร' 351]
 ['2021-04-15' 'กรุงเทพมหานคร' 409]
 ['2021-04-16' 'กรุงเทพมหานคร' 312]]
ขอนแก่น
('สตูล', 'ระนอง')
